In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Before running the following cell create a folder in Shared Drives named DLF2020 and add the DLF2020 (shared by Dhvani) folder's shortcut to it.
Also change runtime to GPU

In [2]:
%cd '/content/drive/Shared drives/DLF2020/DLF2020/Data Collection Module'

/content/drive/.shortcut-targets-by-id/1XCqSJ4wOA_0YGBeiVFwWnOq2NUCpBEMO/DLF2020/Data Collection Module


The following code downloads and builds open pose for current session you need to run it only first time

In [3]:
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
# git_repo_url = 'https://github.com/dinatih/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949

  print("install new CMake becaue of CUDA10")
  if not exists('cmake-3.13.0-Linux-x86_64.tar.gz'):
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("CMakelist.txt's caffe fix")
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


install new CMake becaue of CUDA10
clone openpose
CMakelist.txt's caffe fix
install system dependencies
build openpose
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthrea

The following functions download videos according to a json config file named WLASL_v3.0.json

In [4]:
import os, shutil
import json
import time
import sys
import urllib.request
from multiprocessing.dummy import Pool
import cv2

import random

!pip install youtube-dl

# import logging
# logging.basicConfig(filename='download_{}.log'.format(int(time.time())), filemode='w', level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))


def request_video(url, referer=''):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

    headers = {'User-Agent': user_agent,
               }
    
    if referer:
        headers['Referer'] = referer

    request = urllib.request.Request(url, None, headers)  # The assembled request

    # logging.info('Requesting {}'.format(url))
    response = urllib.request.urlopen(request)
    data = response.read()  # The data you need

    return data


def save_video(data, saveto):
    with open(saveto, 'wb+') as f:
        f.write(data)

    # please be nice to the host - take pauses and avoid spamming
    time.sleep(random.uniform(0.5, 1.5))


def download_youtube(url, dirname, video_id):
    raise NotImplementedError("Urllib cannot deal with YouTube links.")


def download_aslpro(url, dirname, video_id):
    saveto = os.path.join(dirname, '{}.swf'.format(video_id))
    if os.path.exists(saveto):
        # logging.info('{} exists at {}'.format(video_id, saveto))
        return 

    data = request_video(url, referer='http://www.aslpro.com/cgi-bin/aslpro/aslpro.cgi')
    save_video(data, saveto)


def download_others(url, dirname, video_id):
    saveto = os.path.join(dirname, '{}.mp4'.format(video_id))
    if os.path.exists(saveto):
        # logging.info('{} exists at {}'.format(video_id, saveto))
        return 
    
    data = request_video(url)
    save_video(data, saveto)


def select_download_method(url):
    if 'aslpro' in url:
        return download_aslpro
    elif 'youtube' in url or 'youtu.be' in url:
        return download_youtube
    else:
        return download_others

def video_to_frames(video_path, size=None):
    """
    video_path -> str, path to video.
    size -> (int, int), width, height.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if ret:
            if size:
                frame = cv2.resize(frame, size)
            frames.append(frame)
        else:
            break
    cap.release()
    return frames


def convert_frames_to_video(frame_array, path_out, size, fps=25):
    out = cv2.VideoWriter(path_out, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()


def extract_frame_as_video(src_video_path, start_frame, end_frame):
    frames = video_to_frames(src_video_path)
    return frames[start_frame: end_frame+1]


def extract_videos(inst, pre_preprocessed, post_preprocessed):
  # we will call preprcess.py that came with WLASL
  !sh swf2mp4_new.sh          
  url = inst['url']
  video_id = inst['video_id']
  if 'youtube' in url or 'youtu.be' in url:
      yt_identifier = url[-11:]
      src_video_path = os.path.join(pre_preprocessed, yt_identifier + '.mp4')
      dst_video_path = os.path.join(post_preprocessed, video_id + '.mp4')
      if not os.path.exists(src_video_path):
          return
      if os.path.exists(dst_video_path):
          return
      # because the JSON file indexes from 1.
      start_frame = inst['frame_start'] - 1
      end_frame = inst['frame_end'] - 1
      if end_frame <= 0:
          shutil.copyfile(src_video_path, dst_video_path)
          return
      selected_frames = extract_frame_as_video(src_video_path, start_frame, end_frame)
      # when OpenCV reads an image, it returns size in (h, w, c)
      # when OpenCV creates a writer, it requres size in (w, h).
      size = selected_frames[0].shape[:2][::-1]
      convert_frames_to_video(selected_frames, dst_video_path, size)
      
  else:
      src_video_path = os.path.join(pre_preprocessed, video_id + '.mp4')
      dst_video_path = os.path.join(post_preprocessed, video_id + '.mp4')
      if os.path.exists(dst_video_path):
          return
      if not os.path.exists(src_video_path):
          return
      shutil.copyfile(src_video_path, dst_video_path)

def download_nonyt_videos(inst, pre_preprocessed, post_preprocessed, saveto='raw_videos'):
  video_url = inst['url']
  video_id = inst['video_id']
  # logging.info('gloss: {}, video: {}.'.format(gloss, video_id))

  download_method = select_download_method(video_url)    
  
  if download_method != download_youtube:
    try:
        download_method(video_url, saveto, video_id)
        extract_videos(inst,pre_preprocessed,post_preprocessed)
    except Exception as e:
        # logging.error('Unsuccessful downloading - video {}'.format(video_id))
        return

def check_youtube_dl_version():
    ver = os.popen('youtube-dl --version').read()

    assert ver, "youtube-dl cannot be found in PATH. Please verify your installation."
    assert ver >= '2020.03.08', "Please update youtube-dl to newest version."


def download_yt_videos(inst, pre_preprocessed, post_preprocessed,saveto='raw_videos'):
    video_url = inst['url']
    video_id = inst['video_id']

    if 'youtube' not in video_url and 'youtu.be' not in video_url:
        return

    if os.path.exists(os.path.join(saveto, video_url[-11:] + '.mp4')) or os.path.exists(os.path.join(saveto, video_url[-11:] + '.mkv')):
        # logging.info('YouTube videos {} already exists.'.format(video_url))
        return
    else:
        cmd = "youtube-dl \"{}\" -o \"{}%(id)s.%(ext)s\""
        cmd = cmd.format(video_url, saveto + os.path.sep)

        rv = os.system(cmd)
        extract_videos(inst,pre_preprocessed,post_preprocessed)
        if not rv:
            return
            # logging.info('Finish downloading youtube video url {}'.format(video_url))
        else:
            return
            # logging.error('Unsuccessful downloading - youtube video url {}'.format(video_url))

        # please be nice to the host - take pauses and avoid spamming
        time.sleep(random.uniform(1.0, 1.5))

For each word in the json file, the code first clears the directory where videos are stored, download corresponding videos and then calls openpose on all the videos in that directory 

In [ ]:
%tensorflow_version 1.x
import json
import preprocess
indexfile = 'WLASL_v0.3.json'
save_to = 'raw_videos'
pre_preprocessed = 'raw_videos_mp4'
post_preprocessed = 'videos'
character = 'a'
# I am keeping a log file to keep track of the last word that was processed

if __name__ == '__main__':
    content = json.load(open(indexfile))
    for entry in content:
        # first we will delete existing videos in a folder
        if not os.path.exists(save_to):
          os.mkdir(save_to)
        else:  
          try:
            if os.path.isfile(save_to) or os.path.islink(save_to):
                os.unlink(save_to)
            elif os.path.isdir(save_to):
                shutil.rmtree(save_to)
          except Exception as e:
            pass
        if os.path.exists(post_preprocessed):
          try:
            if os.path.isfile(post_preprocessed) or os.path.islink(post_preprocessed):
                os.unlink(post_preprocessed)
            elif os.path.isdir(post_preprocessed):
                shutil.rmtree(post_preprocessed)
          except Exception as e:
            pass
        os.mkdir(post_preprocessed)
        gloss = entry['gloss']
        if gloss[0] == character:  
          print(gloss)
          with open('log.txt','a')as f:
            f.write(gloss + '\n')
          instances = entry['instances']
          for inst in instances:  
            # code to download all kinds of videos
            download_nonyt_videos(inst,pre_preprocessed,post_preprocessed,save_to)
            check_youtube_dl_version()
            download_yt_videos(inst,pre_preprocessed,post_preprocessed,save_to)
            # code to go through the downloaded videos and call openpose
            colab_video_path = os.path.join(post_preprocessed, inst['video_id'] + '.mp4')
            colab_openpose_video_path = colab_video_path.replace('.mp4', '') + '-openpose.mp4'
            if not os.path.exists(colab_openpose_video_path):
              #todo see output file location for json and video
              !cd openpose && ./build/examples/openpose/openpose.bin --hand --face --number_people_max 12 --video '../{colab_video_path}'  --display 0 --write_video '../{colab_openpose_video_path}' --write_json '../pipelineDemo/2DJSON_Keypoints/{inst["video_id"]}' # --net_resolution "-1x736" --scale_number 4 --scale_gap 0.25
              !cd ..
              # after processing a single instances of a single word we will create a h5 for it
              !python3 "./pipelineDemo/pipeline_demo_01_openpose2h5.py"
              # finally we create 3d skeletal structures for all the frames in one video video and store it in a single txt file
              video_id = inst['video_id']
              !python3 "./pipelineDemo/demo.py" $video_id
              data = {}
              data[inst['video_id']] = inst
              with open('Training_Data.json', 'a+') as outfile:
                  json.dump(data, outfile)
          

TensorFlow 1.x selected.
all
mkdir: cannot create directory ‘raw_videos_mp4’: File exists
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libs

In [ ]:
# code to merge 2D features of a video into a single json file and adding a counter to it and convert text files containing 3D keypoints to a single json file
# all the collected data will be stored in the training data Folder in DLF2020
# 
import os
import json

%cd '/content/drive/Shared drives/DLF2020/DLF2020'

# # code to merge multiple 2d json files into one json file
# directory = r'./Data Collection Module/pipelineDemo/2DJSON_Keypoints'
# count_frames = 0
# lines_written = 0
# for video_name in os.listdir(directory):
#     dir = os.path.join(directory, video_name)
#     temp = {}
#     for frame_name in os.listdir(dir):
#       frame_number = frame_name.split('_')[1]
#       f = open(os.path.join(dir,frame_name))
#       temp[int(frame_number)] = json.load(f)
#       count_frames += 1
#     outfile = open("./Training Data/2D Data/"+video_name + '.json','w')
#     print(video_name,count_frames,len(temp.keys()))
#     json.dump(temp, outfile, sort_keys=True)
    
# code to convert 3d txt file to one json file with counter
# I am assuming that the 2d to 3d conversion maintains the order of the frames and so while converting from 2d to 3d I am directly taking line number to frame number
directory = r'./Data Collection Module/Training_Data_3D'
lines_written = 0
for video_name in os.listdir(directory):
    count_frames = 0
    file_name = os.path.join(directory, video_name)
    temp = {}
    f = open(file_name)
    for line in f.readlines():
      temp[count_frames] = line.replace('\t',',').replace('\n','')
      count_frames += 1
      video_name = video_name.replace('txt','json')
    outfile = open("./Training Data/3D Data/"+ video_name,'w')
    json.dump(temp, outfile, sort_keys=True)
    print(video_name)


No need to run following code for each cell but won't harm to check few videos and their corresponding visualizations randomly just change file/folder names

---



The following code just creates a visualization of final 3d data

In [ ]:
# todo need to clean json file so that it only contains uniqueu values after each word is processed
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pipelineDemo.skeletalModel as skeletalModel
file_name = "01995"
f = open("Training_Data_3D/"+file_name+".txt",'r')
l = []
for line in f:
  l.append([float(x) for x in (line.split())])
l = np.array(l)
print(l.shape)
for pose in l[0::2]:
  fig = plt.figure()
  ax  = fig.add_subplot(111, projection = '3d')
  #ax  = fig.add_subplot(111)
  def drawLine(i,j,pose,ax):
    ax.plot([pose[i*3],pose[j*3]],[pose[i*3+2],pose[j*3+2]],[-pose[i*3+1],-pose[j*3+1]], color = 'b')
    #ax.plot([pose[i*3],pose[j*3]],[-pose[i*3+1],-pose[j*3+1]], color = 'b')
  # structure = skeletalModel.getSkeletalModelStructure()
  lines4disp = ((0,1),(1,2),(2,3),(3,4),(1,5),(5,6),(6,7),(7,8),(8,9),(9,10),(10,11),(11,12),(8,13),(13,14),(14,15),(15,16),(8,17),(7,18),(18,19),(19,20),(8,21),(21,22),(22,23),(23,24),(8,25),(25,26),(26,27,(27,28),(4,29),(29,30),(30,31),(31,32),(32,33),29,34),(34,35),(35,36),(36,37),(29,38),(38,39),(39,40),(40,41),(29,42),(42,43),(43,44),(44,45),(29,46),(46,47),(47,48),(48,49))

  for s in lines4disp:
    drawLine(s[0],s[1],pose,ax)
  #ax.scatter(pose[150::2],pose[151::2],[0.2]* 105)

The following code visualizes 2d data

In [ ]:
# following code plots 2d keypoints to check that they are correct
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pipelineDemo.pipelineDemo.skeletalModel as skeletalModel
import json
file_name = "./pipelineDemo/pipelineDemo/2DJSON_Keypoints/65043/65043_000000000026_keypoints.json"
f = open(file_name,'r')
data = json.load(f)
theTallest = data["people"][0]
pointsP = []
pointsLH = []
pointsRH = []
pointsFace = []
for i in range(0,len(theTallest["pose_keypoints_2d"]),3):
  pointsP.append(theTallest["pose_keypoints_2d"][i])
  pointsP.append(theTallest["pose_keypoints_2d"][i+1])

for i in range(0,len(theTallest["hand_left_keypoints_2d"]),3):
  pointsLH.append(theTallest["hand_left_keypoints_2d"][i])
  pointsLH.append(theTallest["hand_left_keypoints_2d"][i+1])


for i in range(0,len(theTallest["hand_right_keypoints_2d"]),3):
  pointsRH.append(theTallest["hand_right_keypoints_2d"][i])
  pointsRH.append(theTallest["hand_right_keypoints_2d"][i+1])

for i in range(0,len(theTallest["face_keypoints_2d"]),3):
  pointsFace.append(theTallest["face_keypoints_2d"][i])
  pointsFace.append(theTallest["face_keypoints_2d"][i+1])
print(len(pointsP))
print(len(pointsLH))
print(len(pointsRH))
print(len(pointsFace))
pose = np.array(pointsP[:16:]+ pointsLH + pointsRH + pointsFace)
print(pose.shape)
fig = plt.figure(figsize=(8, 6),dpi = 90)
ax  = fig.add_subplot(111)
def drawLine(i,j,pose,ax):
  ax.plot([pose[i*2],pose[j*2]],[-pose[i*2+1],-pose[j*2+1]], color = 'b')

# structure = skeletalModel.getSkeletalModelStructure()
lines4disp = ((0,1),(1,2),(2,3),(3,4),(1,5),(5,6),(6,7),(7,8),(8,9),(9,10),(10,11),(11,12),(8,13),(13,14),(14,15),(15,16),(8,17),(7,18),(18,19),(19,20),(8,21),(21,22),(22,23),(23,24),(8,25),(25,26),(26,27,(27,28),(4,29),(29,30),(30,31),(31,32),(32,33),29,34),(34,35),(35,36),(36,37),(29,38),(38,39),(39,40),(40,41),(29,42),(42,43),(43,44),(44,45),(29,46),(46,47),(47,48),(48,49))

for s in lines4disp:
  drawLine(s[0],s[1],pose,ax)

ax.scatter(pose[150::2],-1 * pose[151::2])

In [ ]:
# the following code converts .sh script to proper format 
# !hexdump -C swf2mp4.sh 
# !cat swf2mp4.sh | tr -d '\r' >> yournewscript.sh